# NIfTI Image Slicer with HU Windowing

This notebook visualizes 3D NIfTI images, allowing you to slice through different axes and adjust the Hounsfield Unit (HU) window interactively.

## Required Libraries
Make sure you have the following libraries installed. You can install them using pip:

```bash
pip install nibabel numpy matplotlib ipywidgets
```

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, Dropdown, FloatSlider
from pathlib import Path

## 1. Load NIfTI Data

Specify the path to your NIfTI file. We'll use one of the files we found earlier.

In [2]:
# Define the path to the NIfTI file
# ※パスは適宜変更してください
file_path = Path('/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/data/3d_data/visualization/vol/vol_1003_L1.nii.gz')

# 前処理で使用した正規化の最大値 (Bone WindowのMax)
# 前回のコード設定では 1800 でした。もし1900に変更していた場合はここを1900にしてください。
ORIGINAL_HU_MAX = 1900 

# Load the NIfTI file
try:
    nii_img = nib.load(file_path)
    data = nii_img.get_fdata()
    
    print(f"Successfully loaded: {file_path.name}")
    print(f"Original Data Range: Min={data.min():.4f}, Max={data.max():.4f}")
    
    # 【ここが修正ポイント】
    # 正規化されている(0.0-1.0)データを、元のHUスケール(0-1800)に戻す
    data = data * ORIGINAL_HU_MAX
    
    print(f"Restored HU Range: Min={data.min():.1f}, Max={data.max():.1f}")
    print(f"Image shape: {data.shape}")

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded: vol_1003_L1.nii.gz
Original Data Range: Min=0.0000, Max=1.0000
Restored HU Range: Min=0.0, Max=1900.0
Image shape: (128, 128, 128)


## 2. HU Windowing and Slicing Function

This function applies a window to the HU values of an image slice to improve contrast.

In [3]:
def apply_hu_window(image_slice, window_center, window_width):
    """Apply HU windowing to a 2D slice."""
    min_val = window_center - window_width / 2
    max_val = window_center + window_width / 2
    windowed_slice = np.clip(image_slice, min_val, max_val)
    return windowed_slice

## 3. Interactive Visualization

Use the sliders and dropdown to explore the 3D volume.
- **Slice Index**: Selects the slice number along the chosen axis.
- **View Axis**: Changes the view (Axial, Coronal, Sagittal).
- **HU Window Center**: Sets the center of the HU window (brightness).
- **HU Window Width**: Sets the width of the HU window (contrast).

In [4]:
def interactive_slicer(slice_index, view_axis, hu_center, hu_width):
    """Callback function to display the slice based on widget values."""
    if data is None:
        print("Data not loaded.")
        return

    # Select the slice based on the chosen axis
    if view_axis == 'Axial':
        img_slice = data[:, :, slice_index]
    elif view_axis == 'Coronal':
        img_slice = data[:, slice_index, :]
    elif view_axis == 'Sagittal':
        img_slice = data[slice_index, :, :]
    else:
        return
    
    # Apply the HU window
    windowed_slice = apply_hu_window(img_slice, hu_center, hu_width)

    # Display the image
    plt.figure(figsize=(8, 8))
    plt.imshow(windowed_slice.T, cmap='gray', origin='lower')
    plt.title(f'{view_axis} View, Slice: {slice_index}, HU Center: {hu_center}, HU Width: {hu_width}')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.axis('on')
    plt.show()

# Get dimensions for sliders
max_axial = data.shape[2] - 1
max_coronal = data.shape[1] - 1
max_sagittal = data.shape[0] - 1

# Define widgets
slice_slider = IntSlider(min=0, max=max_axial, step=1, value=max_axial//2, description='Slice Index:', continuous_update=False)
axis_dropdown = Dropdown(options=['Axial', 'Coronal', 'Sagittal'], value='Axial', description='View Axis:')
hu_center_slider = FloatSlider(min=-1000, max=2000, step=10, value=400, description='HU Window Center:', continuous_update=False, readout_format='.0f')
hu_width_slider = FloatSlider(min=1, max=4000, step=10, value=1800, description='HU Window Width:', continuous_update=False, readout_format='.0f')

def update_slice_slider_range(*args):
    """Update the slice slider's max value when the axis changes."""
    if axis_dropdown.value == 'Axial':
        slice_slider.max = max_axial
    elif axis_dropdown.value == 'Coronal':
        slice_slider.max = max_coronal
    else: # Sagittal
        slice_slider.max = max_sagittal
    slice_slider.value = slice_slider.max // 2 # Reset to middle

axis_dropdown.observe(update_slice_slider_range, 'value')

# Create the interactive display
interact(interactive_slicer, slice_index=slice_slider, view_axis=axis_dropdown, hu_center=hu_center_slider, hu_width=hu_width_slider);

interactive(children=(IntSlider(value=63, continuous_update=False, description='Slice Index:', max=127), Dropd…